Методы и алгоритмы обработки слабоструктурированных данных
Лабораторная работа №2 
Болотов М.В. АСУ4-22-1м

In [42]:
import csv                                                                       # Импорт необходимых библиотек
import json  
from bs4 import BeautifulSoup                                                       
import xml.etree.ElementTree as ET

In [43]:
def load_file(file_path):                                                        # Функция для загрузки содержимого файла
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def save_to_file(data, file_path):                                               # Функция для сохранения данных в файл
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(data)

In [44]:
def html_to_json(html_string):                                                   # Функция для преобразования HTML в JSON
    soup = BeautifulSoup(html_string, 'html.parser')                             # Создаем объект BeautifulSoup для парсинга HTML 
    return json.dumps(soup.prettify(), indent=4)                                 # Преобразуем HTML в строку JSON с отступами для удобочитаемости

In [45]:
def html_to_xml(html_string):                                                    # Функция для преобразования HTML в XML
    soup = BeautifulSoup(html_string, 'html.parser')                             # Создаем объект BeautifulSoup для парсинга HTML
    return soup.prettify()                                                       # Преобразуем HTML в строку XML с сохранением форматирования

In [46]:
def xml_to_json(xml_string):                                                     # Функция для преобразования XML в JSON
    root = ET.fromstring(xml_string)                                             # Парсим XML-строку в объект ElementTree
    xml_dict = {root.tag: traverse_xml(root)}                                    # Вызываем рекурсивную функцию traverse_xml для построения словаря из XML
    return json.dumps(xml_dict, indent=4)                                        # Преобразуем словарь в строку JSON с отступами для удобочитаемости

In [47]:
def xml_to_csv(xml_string):                                                      # Функция для преобразования XML в CSV
    root = ET.fromstring(xml_string)                                             # Парсим XML-строку в объект ElementTree
    csv_data = []
    for child in root:                                                           # Обходим дочерние элементы корневого элемента XML
        row = [child.tag] + [elem.text for elem in child]                        # Формируем строку CSV для каждого дочернего элемента
        csv_data.append(row)
    return '\n'.join([','.join(row) for row in csv_data])                        # Соединяем строки CSV в одну строку с разделителем '\n'

In [48]:
def json_to_xml(json_string):                                                    # Функция для преобразования JSON в XML
    json_dict = json.loads(json_string)                                          # Преобразуем строку JSON в словарь
    root_tag = list(json_dict.keys())[0]                                         # Получаем имя корневого элемента из ключа словаря
    root = ET.Element(root_tag)                                                  # Создаем корневой элемент XML
    build_xml(root, json_dict[root_tag])                                         # Вызываем рекурсивную функцию build_xml для построения XML из словаря
    return ET.tostring(root, encoding='unicode')                                 # Преобразуем дерево XML в строку с кодированием в unicode

In [49]:
def json_to_csv(json_string):                                                    # Функция для преобразования JSON в CSV
    json_dict = json.loads(json_string)                                          # Преобразуем строку JSON в список словарей
    header = list(json_dict[0].keys())                                           # Получаем заголовок (названия столбцов) из ключей первого словаря
    rows = [header]                                                              # Формируем строки CSV, включая заголовок и данные
    for item in json_dict:
        rows.append([str(item.get(key, '')) for key in header])
    return '\n'.join([','.join(row) for row in rows])                            # Соединяем строки CSV в одну строку с разделителем '\n'

In [50]:
def csv_to_json(csv_string):                                                     # Функция для преобразования CSV в JSON
    rows = [row.split(',') for row in csv_string.split('\n')]                    # Разбиваем строку CSV на список строк (каждая строка - список значений)
    header = rows[0]                                                             # Получаем заголовок (названия столбцов) из первой строки
    json_data = []                                                               # Формируем список словарей, каждый словарь - одна строка данных
    for row in rows[1:]:
        json_data.append({header[i]: value for i, value in enumerate(row)})
    return json.dumps(json_data, indent=4)                                       # Преобразуем список словарей в строку JSON с отступами для удобочитаемости

In [51]:
def csv_to_xml(csv_string):                                                      # Функция для преобразования CSV в XML
    rows = [row.split(',') for row in csv_string.split('\n')]                    # Разбиваем строку CSV на список строк (каждая строка - список значений)
    root = ET.Element('root')                                                    # Создаем корневой элемент XML
    for row in rows:                                                             # Для каждой строки CSV, кроме заголовка
        parent = ET.SubElement(root, row[0])                                     # Создаем дочерний элемент с именем из первого столбца
        for value in row[1:]:                                                    # Для каждого значения в строке, кроме первого
            child = ET.SubElement(parent, 'item')                                # Создаем дочерний элемент 'item' и присваиваем ему значение
            child.text = value
    return ET.tostring(root, encoding='unicode')                                 # Преобразуем дерево XML в строку с кодированием в unicode

In [52]:
def traverse_xml(element):                                                       # Рекурсивная функция для обхода дерева XML и построения словаря
    xml_dict = {}
    for child in element:                                                        # Для каждого дочернего элемента текущего элемента
        if child.tag in xml_dict:                                                # Если элемент уже присутствует в словаре, добавляем его в список
            if isinstance(xml_dict[child.tag], list):
                xml_dict[child.tag].append(traverse_xml(child))
            else:
                xml_dict[child.tag] = [xml_dict[child.tag], traverse_xml(child)]
        else:                                                                    # Если элемент встречается впервые, добавляем его как значение или текст
            xml_dict[child.tag] = traverse_xml(child) if len(child) > 0 else child.text
    return xml_dict

In [53]:
def build_xml(parent, data):                                                     # Рекурсивная функция для построения XML из словаря
    if isinstance(data, dict):                                                   # Если переданные данные - словарь
        for key, value in data.items():                                          # Для каждого ключа и значения в словаре
            if isinstance(value, list):                                          # Если значение - список, добавляем дочерние элементы
                for item in value:
                    child = ET.SubElement(parent, key)
                    build_xml(child, item)
            else:                                                                # Если значение не список, добавляем его как дочерний элемент
                child = ET.SubElement(parent, key)
                build_xml(child, value)
    else:                                                                        # Если переданные данные - не словарь, добавляем текст в качестве содержимого
        parent.text = data

In [54]:
# Пример использования:

html_content = load_file(r"C:\Users\79125\OneDrive\Рабочий стол\Учёба ПНИПУ АСУ\4 семестр\Методы и алгоритмы обработки слабоструктурированных данных\lr2\sample1.html")                                                    # Загрузка содержимого HTML файла

json_result = html_to_json(html_content)                                                   # Преобразование HTML в JSON
print("HTML to JSON:")
print(json_result)

save_to_file(json_result, r"C:\Users\79125\OneDrive\Рабочий стол\Учёба ПНИПУ АСУ\4 семестр\Методы и алгоритмы обработки слабоструктурированных данных\lr2\output.json")                                                  # Сохранение результата в JSON файл

xml_result = html_to_xml(html_content)                                                     # Преобразование HTML в XML
print("\nHTML to XML:")
print(xml_result)

save_to_file(xml_result, r"C:\Users\79125\OneDrive\Рабочий стол\Учёба ПНИПУ АСУ\4 семестр\Методы и алгоритмы обработки слабоструктурированных данных\lr2\output.xml")                                                   # Сохранение результата в XML файл

HTML to JSON:
"<!DOCTYPE html>\n<html>\n <head>\n  <script charset=\"UTF-8\" src=\"/FD126C42-EBFA-4E12-B309-BB3FDD723AC1/main.js?attr=qX09-MqOZbO0YYlRBocFA_H8-C2GcKnOZ2dxO389GFY02ymzVsaurTFDnbhmeL9cIn9XPWafCqT3-Hdgv--6vOf4-JfEzntG1aP137Hl2Ek\" type=\"text/javascript\">\n  </script>\n  <link crossorigin=\"anonymous\" href=\"/E3E8934C-235A-4B0E-825A-35A08381A191/abn/main.css?attr=aHR0cHM6Ly9maWxlc2FtcGxlcy5jb20vc2FtcGxlcy9jb2RlL2h0bWwvc2FtcGxlMS5odG1s\" rel=\"stylesheet\"/>\n  <script data-cfasync=\"false\" src=\"https://privacy.gatekeeperconsent.com/tcf2_stub.js\">\n  </script>\n  <script>\n   var _ez_ab_test = 'mod33'\n  </script>\n  <script>\n   var __ezHttpConsent={setByCat:function(src,tagType,attributes,category,force){var setScript=function(){if(force||window.ezTcfConsent[category]){var scriptElement=document.createElement(tagType);scriptElement.src=src;attributes.forEach(function(attr){for(var key in attr){if(attr.hasOwnProperty(key)){scriptElement.setAttribute(key,attr[key]);}}}